# NYSE DataFrames and Databases

In this activity, you will read, clean and load data to a database table. 

## Instructions

### Create the Pandas DataFrame

1. Import the modules for Pandas, the Path, and SQLAlchemy.
2. Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame called `nyse_df`. Review the DataFrame.
3. Check the data types (`dtypes`) of each column in the DataFrame.

### Clean the Pandas DataFrame

1. Apply the provided `clean_currency` function to the 'MarketCap' column of the `nyse_df` DataFrame.

  * The `clean_currency` function performs the following actions:
    -Removes non-numeric characters (ie '\$', 'M', and 'B') from the data entry
    -Converts the value to a float
    -Scales the data according to the suffixes 'M' and 'B'
    -Returns the cleaned and scaled data entry

2. Review the contents of the 'MarketCap' column to confirm that data has been cleaned as expected.

### Load the DataFrame into a SQLite Database

1. Create a database connection string that loads the SQLite database called `nyse.db` into the Resources folder.
2. Create a [database engine](https://docs.sqlalchemy.org/en/14/core/engines.html) called `nyse_engine`. Be sure to include the `echo=True` parameter.
3. Using the Pandas [`to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function, write the `nyse_df` DataFrame to a table called `NYSE` in the `nyse.db`. Be sure to include the parameters for `index` and `if_exists`.
4. Get the table name from the database to confirm it was created.

### Query the SQL Database

1. Write a SQL query to select all companies in the Technology sector AND in the Software industry AND that have a market cap greater than 30 billion USD.
2. Using the Pandas [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) execute the SQL query and assign the results to a new Pandas DataFrame called `nyse_tech_companies_df`.
3. Review the DataFrame.



## Create the Pandas DataFrame

### 1. Import the modules for Pandas, the Path, and SQLAlchemy.

In [18]:
# Import modules
import pandas as pd
from pathlib import Path
import sqlalchemy as sql


### 2. Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame called `nyse_df`. Review the DataFrame.

In [19]:
# Read the `nyse_companylist.csv` file from the Resources folder into a Pandas DataFrame 
nyse_df = pd.read_csv(
    Path('../Resources/nyse_companylist.csv')
)

# Review the DataFrame
nyse_df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,4.79,$580.24M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,161.66,$93.12B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.94,$126.42M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,15.71,$1.64B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.39,$33.19M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


### 3. Review the data types (`dtypes`) of each column in the DataFrame.

In [20]:
# Check dtypes of dataframe
nyse_df.dtypes

Symbol            object
Name              object
LastSale         float64
MarketCap         object
IPOyear          float64
Sector            object
industry          object
Summary Quote     object
Unnamed: 8       float64
dtype: object

## Clean the Pandas DataFrame

### 1. Apply the provided `clean_currency` function to the 'MarketCap' column of the `nyse_df` DataFrame. 

The `clean_currency` function performs the following actions:
   - Removes non-numeric characters (ie '$', 'M', and 'B') from the data entry
   - Converts the value to a float
   - Scales the data according to the suffixes 'M' and 'B'
   - Returns the cleaned and scaled data entry

In [21]:
# This function converts the string values into a floating point number
def clean_currency(price_string):
    price = price_string
    if type(price_string) == str:
        if price_string[-1] == 'M':
            million = 1000000
            price_string = price_string.replace('$', '')
            price_string = price_string.replace('M', '')
            price = float(price_string)
            price = price * million
        else:
            billion = 1000000000
            price_string = price_string.replace('$', '')
            price_string = price_string.replace('B', '')
            price = float(price_string)
            price = price * billion
    return price



In [22]:
# Apply the clean_currency function to MarketCap column
nyse_df['MarketCap'] = nyse_df['MarketCap'].apply(clean_currency)

### 2. Review the contents of the 'MarketCap' column.

In [23]:
# Review the contents for the MarketCap column
nyse_df['MarketCap'][0:5]

0    5.802400e+08
1    9.312000e+10
2    1.264200e+08
3    1.640000e+09
4    3.319000e+07
Name: MarketCap, dtype: float64

## Load the DataFrame into a SQLite Database

### 1. Create a database connection string that loads the SQLite database called `nyse.db` into the Resources folder.

In [24]:
# Create a database connection string
nyse_connection_string = 'sqlite:///../Resources/nyse.db'

### 2. Create a [database engine](https://docs.sqlalchemy.org/en/14/core/engines.html) called `nyse_engine`. Be sure to include the `echo=True` parameter.

In [25]:
# Create a database engine
nyse_engine = sql.create_engine(nyse_connection_string, echo=True)

### 3. Using the Pandas [`to_sql`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) function, write the `nyse_df` DataFrame to a table called `NYSE` in the `nyse.db`.

In [26]:
# Add the cleaned data to a table named NYSE in the database.
nyse_df.to_sql('NYSE', nyse_engine, index=False, if_exists='replace')

2021-04-14 12:08:50,643 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-04-14 12:08:50,644 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 12:08:50,645 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-04-14 12:08:50,646 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 12:08:50,647 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("NYSE")
2021-04-14 12:08:50,648 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 12:08:50,648 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("NYSE")
2021-04-14 12:08:50,649 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 12:08:50,653 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "NYSE" (
	"Symbol" TEXT, 
	"Name" TEXT, 
	"LastSale" FLOAT, 
	"MarketCap" FLOAT, 
	"IPOyear" FLOAT, 
	"Sector" TEXT, 
	industry TEXT, 
	"Summary Quote" TEXT, 
	"Unnamed: 8" FLOAT
)


2021-04-14 12:08:50,653 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 12:08

### 4. Get the table name from the database to confirm it was created.

In [27]:
# Get the table names from the database
nyse_engine.table_names()

2021-04-14 12:08:51,234 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-04-14 12:08:51,235 INFO sqlalchemy.engine.base.Engine ()


['NYSE']

## Query the SQL Database

### 1. Write a SQL query to select all companies in the Technology sector AND in the Software industry AND that have a market cap greater than 30 billion USD.

In [10]:
# Select all stocks where the industry is related to Software in the Technology sector having a market cap greater than 30,000,000,000.
sql_query = """
SELECT *
FROM NYSE
WHERE Sector = 'Technology'
AND Industry LIKE '%Software%'
AND MarketCap > 30000000000
"""

### 2. Using the Pandas [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html) execute the SQL query and assign the results to a new Pandas DataFrame called `nyse_tech_companies_df`.

In [11]:
# Execute the SQL query and read the data into a Pandas DataFrame
nyse_tech_companies_df = pd.read_sql_query(sql_query, nyse_engine, index_col='Symbol')

2021-04-14 11:45:43,643 INFO sqlalchemy.engine.base.Engine 
SELECT *
FROM NYSE
WHERE Sector = 'Technology'
AND Industry LIKE '%Software%'
AND MarketCap > 30000000000

2021-04-14 11:45:43,644 INFO sqlalchemy.engine.base.Engine ()


### 3. Review the DataFrame.

In [12]:
# Review the DataFrame
nyse_tech_companies_df.head()

,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
Symbol,,,,,,,,
INFO,IHS Markit Ltd.,80.06,3.177000e+10,1996.0,Technology,"Computer Software: Programming, Data Processing",https://old.nasdaq.com/symbol/info,None
ORCL,Oracle Corporation,59.58,1.793900e+11,1986.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/orcl,None
CRM,Salesforce.com Inc,246.67,2.244700e+11,2004.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/crm,None
SAP,SAP SE,157.08,1.875000e+11,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/sap,None
NOW,"ServiceNow, Inc.",484.21,9.287000e+10,2012.0,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/now,None
